In [ ]:
##### General ToDo list:

# For select_data() function, add "STAGE/ROUTE" selection
#     Add an ooption to view screening 'STAGE/ROUTE' only as well

#//ToDo: write a function to fliter out only singificant breaches/ significant number of cases pe trust etc

# Select_data() is now redundant with filter_data()

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go


In [ ]:
from data_wrangling import get_national
from cancer_plotting import plot_stacked_referrals

In [ ]:
# code example being used - note will need to run the national_data code first 
national_data_link = r'https://www.england.nhs.uk/statistics/wp-content/' \
    + 'uploads/sites/2/2023/12/' \
    + 'CWT-CRS-National-Time-Series-Oct-2009-Oct-2023-with-Revisions.xlsx'

national_data = get_national(national_data_link)
labels = ['Within Standard_28', 'Outside Standard_28', 'Within Standard_31', 'Outside Standard_31'] 
data_to_plot=['Month','Within Standard_28', 'Outside Standard_28', 'Within Standard_31', 'Outside Standard_31']
fig, ax = plot_stacked(national_data[data_to_plot],labels,"Patients")

In [1]:
from data_wrangling import get_provider_data, nhs_code_link, help_with, read_icb_sicb_coding
from data_wrangling import select_months, select_org, select_cancer, select_standard
from data_wrangling import select_data
from data_wrangling import proportion_breaches

from data_wrangling import filter_data

from cancer_plotting import breaches_animated_plot


In [2]:
data = get_provider_data()

In [ ]:
x = select_cancer(data, cancer_type=['whatevs','Unsuspected_breast_ca', 'All_Cancers'])

In [4]:
filter_data(data, {'org':['R0A'], 'standard':'FDS','treatment':['radiotherapy','not_applicable_FDS'],})

Value 'radiotherapy' not in df, continuing without it


,standard,org_code,stage_or_route,treatment_modality,cancer_type,total,within_standard,breaches
month,,,,,,,,
2022-04-01,28-day FDS,R0A,breast_symptom_non_cancer,not_applicable_FDS,Unsuspected_breast_ca,382,144,238
2022-04-01,28-day FDS,R0A,screening,not_applicable_FDS,Invalid,143,67,76
2022-04-01,28-day FDS,R0A,urgent_suspected_cancer,not_applicable_FDS,Suspected_breast_ca,1135,361,774
2022-04-01,28-day FDS,R0A,urgent_suspected_cancer,not_applicable_FDS,Suspected_children_ca,3,1,2
2022-04-01,28-day FDS,R0A,urgent_suspected_cancer,not_applicable_FDS,Suspected_gynecological_ca,287,157,130
...,...,...,...,...,...,...,...,...
2023-03-01,28-day FDS,R0A,urgent_suspected_cancer,not_applicable_FDS,Suspected_sarcoma,71,45,26
2023-03-01,28-day FDS,R0A,urgent_suspected_cancer,not_applicable_FDS,Suspected_skin_ca,403,334,69
2023-03-01,28-day FDS,R0A,urgent_suspected_cancer,not_applicable_FDS,Suspected_testicular_ca,4,3,1


In [ ]:
data.head()

In [ ]:
select_org(data, ['R0A', 'R0D'])

In [ ]:
#data = proportion_breaches((data))

In [ ]:
select_months(data, end_date='2023-10-01')

In [ ]:
help(filter_data)

In [ ]:
filter_data(df=data, filters={'start_month':'05-2022',
                              'end_month':'05-2022',
                              'standard':'FDS'})

In [ ]:
help(select_standard)

In [ ]:
isinstance(np.empty(shape=2), list)

In [ ]:
help_with('standards')

In [ ]:
link_data = nhs_code_link()
# overlap = list(set(data['ORG CODE']) & set(link_data['ORG CODE']))
# unmatched = data[~data['ORG CODE'].astype(str).isin(link_data['ORG CODE'].astype(str))]

In [ ]:
#plt.figure(figsize=[10,10])
sns.stripplot(x='PERIOD', y = 'BREACHES', data=select_org(df=data,org_str='R1K'), hue = 'CANCER_TYPE', legend=False)

In [ ]:
sns.lineplot(x='PERIOD', y='BREACHES', data=select_org(data,'R0A').groupby(by='PERIOD').sum())

In [ ]:
data

In [ ]:
# Need to add informative title displaying type of cancer and full name of organisation + national standard on the plot
# Need to handle if some orgs do not have certain cancers/standards

trial = select_data(data,[('cancer',3), ('org', 'r0a'), ('standard', 'FDS')])
trial = trial.loc[trial['STAGE/ROUTE'] =='NATIONAL SCREENING PROGRAMME']
breaches_animated_plot(trial,[])


In [ ]:
def create_lookup_dict_icb():
    
    icb_codes = read_icb_sicb_coding()
    nhs_link = nhs_code_link()
    
    #Creates dictionary of ICB 3-digit code to ICB Name
    icb_code_to_names = dict(zip(icb_codes['ICB22CDH'], icb_codes['ICB22NM']))
    
    #Creates dictionary of NHS Trust Org code to Higher Level Health Geography
    org_to_hlhg = dict(zip(nhs_link['ORG_CODE'], nhs_link['Higher Level Health Geography']))
    
    return icb_code_to_names, org_to_hlhg
    

In [ ]:
######## Below is work in progress on creating a map with statistics

In [ ]:
def select_to_plot(data, filter_to_apply=[], gdf = gdf):
    
    filtered_df = select_data(data, filter_to_apply)
    
    
    icb_code_to_names, org_to_hlhg = create_lookup_dict_icb()
    
    
    filtered_df['hlhg'] = filtered_df['ORG_CODE'].map(org_to_hlhg)
    filtered_df['ICB23NM'] = filtered_df['hhlg'].map(icb_code_to_names)
    
    labels_for_plotting = {'cancer_type':filtered_df.CANCER_TYPE.unique(),
                           'period':filtered_df.PERIOD.unique(),
                           'standard':filtered_df.STANDARD.unique()
                          }
    
    icb_breaches = round(filtered_df.groupby('ICB23NM').BREACHES.sum()
                         / filtered_df.groupby('ICB23NM').TOTAL.sum(), 2
                        ).astype(float)
    
    
    icb_breaches.rename('proportion_breaches', inplace=True)
    
    merged_gdf = pd.merge(gdf, icb_breaches, left_on='ICB23NM', right_index=True)
    
    return merged_gdf, labels_for_plotting
    
    
    
    

In [ ]:
help(select_to_plot)

In [ ]:
labels = select_to_plot(data, [('cancer',10), ('standard', 'FDS')])[1]
plotting_material = select_to_plot(data, [('cancer',10), ('standard', 'FDS')])[0]

In [ ]:
labels

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
plotting_material.plot(column = 'proportion_breaches',
                           ax=ax, legend=True,
                           legend_kwds={'label': "Proportion Breaches", 'orientation': "vertical"})

plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)


In [ ]:
icb_codes = read_icb_sicb_coding()
nhs_link = nhs_code_link()

icb_code_to_names = dict(zip(icb_codes['ICB22CDH'], icb_codes['ICB22NM']))
org_to_icb = dict(zip(nhs_link['ORG_CODE'], nhs_link['Higher Level Health Geography']))

data['hhlg'] = data['ORG_CODE'].map(org_to_icb)
data['ICB23NM'] = data['hhlg'].map(icb_code_to_names)

In [ ]:
icb_breaches = round(data.groupby('ICB23NM').BREACHES.sum() / data.groupby('ICB23NM').TOTAL.sum(), 2).astype(float)
icb_breaches.rename('proportion_breaches', inplace=True)

In [ ]:
import geopandas as gpd
path_to_shapefile = 'data/ons_shapefile/Integrated_Care_Boards_April_2023_EN_BFC_1659257819249669363/'

In [ ]:
gdf = gpd.read_file(path_to_shapefile)

In [ ]:
gdf.head()

In [ ]:
merged_gdf = pd.merge(gdf, icb_breaches, left_on='ICB23NM', right_index=True)
merged_gdf.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
merged_gdf.plot(column='LAT', ax=ax, legend=True,
                legend_kwds={'label': "Proportion Breaches", 'orientation': "vertical"})

plt.show()